# December 2017: Advent of Code Solutions
## Daniel Näslund
From Dec. 1 to Dec. 25, [I](dannas.name) will be solving the puzzles that appear each day at [Advent of Code](http://adventofcode.com/). The two-part puzzles are released at midnight EST (6:00AM CET); points are awarded to the first 100 people to solve the day's puzzles. 

To understand the problems completely, you will have to read the full description in the "[Day 1](http://adventofcode.com/2017/day/1):" link in each day's section header.

##  Prelude
Here I import common functions and modules so I don't have to do it each day.
These are borrowed from [Peter Norvigs Advent of Code solutions](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb) from 2016. I've also reused his ipython notebook layout.

In [25]:
# Python 3.x
from itertools import islice, cycle, permutations
import operator

cat = ''.join
def Input(day):
    filename = 'advent2017/input{}.txt'.format(day);
    return open(filename);
    # TODO(dannas): Fetch the files from elsewhere to allow remote access

def quantify(iterable, pred=bool):
    return sum(map(pred, iterable))
    
# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors8(point):
    "The eight neighbors (with diagonals)"
    x, y =  point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
           (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha
Given a file of digits, find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

We need to parse with one token lookahead, it's enough to append the first digit to the end of the list for handling the circular case.

In [3]:
def pairs(seq):
    return zip(seq[:-1], seq[1:])

def solve_captcha(str):
    digits = [int(d) for d in str] 
    if len(digits) < 2:
        return 0
    digits.append(digits[0])
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())

1251

In **part 2** we shall compare the digit halfway around the circular list to the current one. The list has an even number of elements.

This require N/2 token lookahead instad of one. I could have appended the first half of the list to the end, but instead I opted for a circular queue. A circular queue can be implicitely represented using indexes that wrap around; or explicitely using an [ADT](https://docs.python.org/3.6/library/collections.html?highlight=deque#collections.deque.rotate); or using operations on iterators. I choose the later.

The [cycle()](https://docs.python.org/3.6/library/itertools.html#itertools.cycle) function provides an iterator to an infinite circular representation of the list. With [islice](https://docs.python.org/3.6/library/itertools.html#itertools.islice) I can select my start and end position in that list.

In [4]:
def pairs(seq):
    N = len(seq)
    half = int(N/2)
    x = islice(cycle(seq), 0, N)
    y = islice(cycle(seq), half, N + half)
    return zip(x, y)

def solve_captcha(str):
    digits = [int(d) for d in str] 
    return sum(x for x, y in pairs(digits) if x == y)
    
solve_captcha(Input(1).read().strip())


1244

## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum
For each row in a spreadsheet, determine the difference between largest and smallest value; the checksum is the sum of all of these differences.

I initially had some trouble parsing the input into a list of list of integers. It's been a while since I used Python.

In [5]:
def maxmins(spreadsheet):
    for row in spreadsheet:
        yield max(row), min(row)

def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x-y for x,y in maxmins(spreadsheet))

41887

In **part 2** we're asked to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. Find those numbers on each line, divide them, and add up each line's result.

In [6]:
def evens(spreadsheet):
   for row in spreadsheet:
        for x, y in permutations(row, 2):
            if x % y == 0:
                yield x, y
                
def parse(line):
    return tuple(int(x) for x in line.split())

spreadsheet = [parse(line) for line in Input(2)]
sum(x/y for x,y in evens(spreadsheet))


226.0

## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory
Walk a a grid in a spiral pattern, a specified number of steps. Then calculate the manhattan distance to the origin.

The trace of steps will form a nested set of cubes. I tried to come up with a neat formula for describing how the the number of points in those cubes increases. Failed. Instead I experimented on paper and found that we increase the sides when we turn East and West.

I decided to represent the points using x,y tuples. I considered using complex numbers first. The walk() function was originally a long list of explicit steps. I then extracted the common parts and ended up with the move and turn functions.

Representing the direction as vectors (2 element tuples of x and y) and using vector addition was a heureka moment for me.

In [5]:
E, N, W, S = [(1, 0), (0, 1), (-1, 0), (0, -1)]    

def move(pos, direction):
    return (X(pos) + X(direction), Y(pos) + Y(direction))

def turn(direction, num_steps):
    changes = {
        E : (N, 0),
        N : (W, 1),
        W : (S, 0),
        S : (E, 1),
    }
    d, delta = changes[direction]
    return d, num_steps + delta


def walk(total_steps):
    pos = (0, 0)
    num_steps = 1
    walked = 0
    direction = E
    
    while total_steps > 0:
        walked += 1
        pos = move(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0

        total_steps -= 1
    return pos

pos = walk(289326 - 1)

cityblock_distance(pos)

419

In **part 2** we're asked to, for each field, store the sum of previously visited fields, including diagonals. The center point has value 1. Once a field is written, its value doesn't change. 

I spent a lot of time trying to come up with a clever algorithm for determining which neighbors to a field was already visited. Then I had an euphyfany and realized that I could compare all 8 possible adjacent fields, if I had stored a mapping between visited fields and their sums. Determine the first value written that is larger than the puzzle input.

In [10]:
square_values = {}

def neighbour_sum(pos, direction):
    return sum(square_values[N] for N in neighbors8(pos) if N in square_values)
   
def walk(lower_limit):
    pos = (0, 0)
    square_values[pos] = 1
    num_steps = 1
    walked = 0
    direction = E
    
    while square_values[pos] <= lower_limit:
        walked += 1
        pos = move(pos, direction)
        square_values[pos] = neighbour_sum(pos, direction)
        if walked == num_steps:
            direction, num_steps = turn(direction, num_steps)
            walked = 0
    return square_values[pos]

walk(289326)

295229

## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases
How many pass phrases are valid, a.k.a. do not contain duplicated words?

In [17]:
lines = [L for L in Input(4)]

def is_valid(line):
    words = line.split()
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0 
    for line in lines)

325

In **part 2** we're asked how many passphrases are valid, this time with the added requirement that no words in the passphrase can be an anagram of another word.

I remember this one from the chapter Aha! Algorithms in the book Programming Pearls. In that book, Jon meant that detecting anagrams is an example of an algorithm that people may bank their head against the wall with until they finally hits the insight. Thankfully, I was already familiar with it.

In [18]:
def is_valid(line):
    words = [cat(sorted(w)) for w in line.split()]
    return len(words) == len(set(words))

sum(1 if is_valid(line) else 0
     for line in lines)

119

## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike
Given a list of relative jumps, determine the number of steps neccessary to reach a destination outside the list. There's a quirk: The jump instructions increment by one upon being visited.

I first wrote this as a straightforward loop. Had one bug: I interpreted the jumps as absolute positions at first. I'm borrowing the quantify function which I found in Peter Norvigs 2017 Advent of Code solution Jupyter notebook.

In [33]:
def jumps(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        pos += instr[pos]
        instr[old] += 1
    yield pos
        
quantify(jumps([int(x) for x in Input(5)]))

375042

In **part 2** we're asked to again calculate number of steps. After each jump, if the offset was 3 or more, decrement it by 1 else increment it by 1.

I wrote jumps2 as a function that returned the number of steps. Then I rewrote it to yield values instead. That caused the running time to increase from "almost instant" to 8s. Doh. Keeping it as-is, since I accidentily removed the original solution (and I don't understand how  Jupyter handles editing history - I want my vi editor instead of  this webbrowser madness!)

In [39]:
def jumps2(instr):
    pos = 0
    while pos >= 0 and pos < N:
        old = pos
        yield pos
        offset = instr[pos]
        pos += instr[pos]
        if offset >= 3:
            instr[old] -= 1
        else:
            instr[old] += 1
    yield pos

quantify(jumps2([int(x) for x in Input(5)]))

28707598

In [40]:
%timeit quantify(jumps2([int(x) for x in Input(5)]))

1 loop, best of 3: 8.16 s per loop
